In [10]:
# Dependancies
import pandas as pd
import os

# File load
file1 = os.path.join('data', 'fatal-police-shootings-data.csv' )
file2 = os.path.join('data', 'ACSDP1Y2015.csv' )
file3 = os.path.join('data', 'ACSDP1Y2016.csv' )
file4 = os.path.join('data', 'ACSDP1Y2017.csv' )
file5 = os.path.join('data', 'ACSDP1Y2018.csv' )

# File read
shootings_df = pd.read_csv(file1)
# c_2015 = pd.read_csv(file2, header=None)
# c_2016 = pd.read_csv(file3, header=None)
# c_2017 = pd.read_csv(file4, header=None)
# c_2018 = pd.read_csv(file5, header=None)
c_2015 = pd.read_csv(file2)
c_2016 = pd.read_csv(file3)
c_2017 = pd.read_csv(file4)
c_2018 = pd.read_csv(file5)

In [11]:
c_2015.head()

,GEO_ID,NAME,DP05_0001E,DP05_0001M,DP05_0001PE,DP05_0001PM,DP05_0002E,DP05_0002M,DP05_0002PE,DP05_0002PM,...,DP05_0082PM,DP05_0083E,DP05_0083M,DP05_0083PE,DP05_0083PM,DP05_0084E,DP05_0084M,DP05_0084PE,DP05_0084PM,DP05_0053PE
0,id,Geographic Area Name,Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Percent!!SEX AND AGE!!Total population,Percent Margin of Error!!SEX AND AGE!!Total po...,Estimate!!SEX AND AGE!!Total population!!Male,Margin of Error!!SEX AND AGE!!Total population...,Percent!!SEX AND AGE!!Total population!!Male,Percent Margin of Error!!SEX AND AGE!!Total po...,...,"Percent Margin of Error!!CITIZEN, VOTING AGE P...","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citi...","Margin of Error!!CITIZEN, VOTING AGE POPULATIO...","Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...","Percent Margin of Error!!CITIZEN, VOTING AGE P...","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citi...","Margin of Error!!CITIZEN, VOTING AGE POPULATIO...","Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...","Percent Margin of Error!!CITIZEN, VOTING AGE P...",Percent!!RACE!!Two or more races
1,0400000US01,Alabama,4858979,*****,4858979,(X),2352810,5380,48.4,0.1,...,(X),1733691,4643,47.5,0.1,1919690,4241,52.5,0.1,1.8
2,0400000US02,Alaska,738432,*****,738432,(X),388695,1732,52.6,0.2,...,(X),282486,1893,53.5,0.3,245762,1773,46.5,0.3,8.6
3,0400000US04,Arizona,6828065,*****,6828065,(X),3387385,2294,49.6,0.1,...,(X),2315216,8813,49.2,0.1,2395232,8328,50.8,0.1,3.3
4,0400000US05,Arkansas,2978204,*****,2978204,(X),1463576,3921,49.1,0.1,...,(X),1053394,4088,48.2,0.1,1132330,3539,51.8,0.1,2.2


In [15]:
# Census Data Cleaning

# Function Setup
# Column pull and format function
def col_pf(df, year):
    # Pulled geo ID, name, and population columns
    df = df.loc[:, ['GEO_ID', 'NAME', 'DP05_0001E']]
    # Renamed columns
    df = df.rename(columns={'NAME': 'Place/State', 'DP05_0001E': f'{year} Population'}) 
    # Split the Place/State column into two new columns
    df[['Place', 'State']] = df['Place/State'].str.rsplit(',', expand=True)
    # Removed rows that only have states (no 'place')
    df = df[53:]
    # Reorganized DF for Population to be at the end
    df = df[['GEO_ID', 'Place/State', 'Place', 'State', f'{year} Population']]
    # Reset index and remove the prior index
    df = df.reset_index(drop=True)
    # Used regex on the Place column to leave the place name without an 
    # additional description
    df[['Place']] = df[['Place']].replace(regex=[r'city$'], value='')
    df[['Place']] = df[['Place']].replace(regex=[r'town$'], value='')
    df[['Place']] = df[['Place']].replace(regex=[r'village$'], value='')
    df[['Place']] = df[['Place']].replace(regex=[r'CDP$'], value='')
    df[['Place']] = df[['Place']].replace(regex=[r'municipality$'], value='')
    df[['Place']] = df[['Place']].replace(regex=[r'zona urbana$'], value='')
    return df

# DF merge function
def col_merge(df, df2, df3, df4):
    merged_df = pd.merge(df, df2, on=('GEO_ID', 'Place/State', 'Place', 'State'), how='outer')
    merged_df = pd.merge(merged_df, df3, on=('GEO_ID', 'Place/State', 'Place', 'State'), how='outer')
    merged_df = pd.merge(merged_df, df4, on=('GEO_ID', 'Place/State', 'Place', 'State'), how='outer')
    return merged_df

In [16]:
# Applied col_pf function on Census DFs
df_2015 = col_pf(c_2015, 2015)
df_2016 = col_pf(c_2016, 2016)
df_2017 = col_pf(c_2017, 2017)
df_2018 = col_pf(c_2018, 2018)

df_2015.head()

,GEO_ID,Place/State,Place,State,2015 Population
0,1600000US0107000,"Birmingham city, Alabama",Birmingham,Alabama,214911
1,1600000US0121184,"Dothan city, Alabama",Dothan,Alabama,67536
2,1600000US0135896,"Hoover city, Alabama",Hoover,Alabama,84839
3,1600000US0137000,"Huntsville city, Alabama",Huntsville,Alabama,189114
4,1600000US0150000,"Mobile city, Alabama",Mobile,Alabama,194305


In [17]:
# Merged the DFs
merged_df = col_merge(df_2015, df_2016, df_2017, df_2018)
merged_df

,GEO_ID,Place/State,Place,State,2015 Population,2016 Population,2017 Population,2018 Population
0,1600000US0107000,"Birmingham city, Alabama",Birmingham,Alabama,214911,213434,212744,209294
1,1600000US0121184,"Dothan city, Alabama",Dothan,Alabama,67536,67714,67526,67814
2,1600000US0135896,"Hoover city, Alabama",Hoover,Alabama,84839,84943,84937,85115
3,1600000US0137000,"Huntsville city, Alabama",Huntsville,Alabama,189114,196225,195369,199808
4,1600000US0150000,"Mobile city, Alabama",Mobile,Alabama,194305,192895,190273,189570
...,...,...,...,...,...,...,...,...
629,1600000US2524960,"Framingham city, Massachusetts",Framingham,Massachusetts,NaN,NaN,NaN,73123
630,1600000US2735180,"Lakeville city, Minnesota",Lakeville,Minnesota,NaN,NaN,NaN,65871
631,1600000US4944320,"Lehi city, Utah",Lehi,Utah,NaN,NaN,NaN,66029
632,1600000US5357535,"Redmond city, Washington",Redmond,Washington,NaN,NaN,NaN,67680


In [18]:
# Count non-NA items in each column
merged_df.count()

GEO_ID             634
Place/State        634
Place              634
State              634
2015 Population    596
2016 Population    605
2017 Population    614
2018 Population    630
dtype: int64

In [19]:
# Drop NA cells
clean_df = merged_df
clean_df.dropna(inplace = True) 
clean_df.count()

GEO_ID             592
Place/State        592
Place              592
State              592
2015 Population    592
2016 Population    592
2017 Population    592
2018 Population    592
dtype: int64

In [20]:
# Type check on DF
# Identified casting will be needed
clean_df.dtypes

GEO_ID             object
Place/State        object
Place              object
State              object
2015 Population    object
2016 Population    object
2017 Population    object
2018 Population    object
dtype: object